In [80]:
from warnings import filterwarnings

filterwarnings("ignore")

import pandas as pd
import numpy as np

from modules.indicators import fibonacci_moving_average, fibonacci_distance_indicator
from modules.backtester import long_only_backtester
from modules.data_fetcher import download_historical_data


In [81]:
symbol = "KDA-USDT" # Best: BNB, BTC, 

df_BTC = download_historical_data(symbol, "4hour")  # .loc["2021-11-20":]
df_BTC.dropna(inplace=True)
# df_BTC = df_BTC.asfreq("H").ffill()
print(df_BTC.shape)
df_BTC.head()


(4022, 7)


,Timestamp,Open,Close,High,Low,Amount,Volume
Date,,,,,,,
2021-05-11 10:00:00,1.620720e+09,1.0000,1.8699,88.0000,1.0000,3.136879e+06,6.180474e+06
2021-05-11 14:00:00,1.620734e+09,1.8699,1.5069,1.8700,1.4991,1.245498e+06,2.148170e+06
2021-05-11 18:00:00,1.620749e+09,1.5069,1.4111,1.5357,1.3527,9.299142e+05,1.330998e+06
2021-05-11 22:00:00,1.620763e+09,1.4207,1.3715,1.4207,1.3220,3.369282e+05,4.629575e+05
2021-05-12 02:00:00,1.620778e+09,1.3705,1.4107,1.4286,1.3513,3.140290e+05,4.346453e+05


In [82]:
df_BTC["FibEMA"] = fibonacci_moving_average(df_BTC.Close)
df_BTC["Fib_distance"] = fibonacci_distance_indicator(df_BTC.Close, df_BTC.FibEMA)

df_BTC.dropna(inplace=True)
df_BTC.head()


,Timestamp,Open,Close,High,Low,Amount,Volume,FibEMA,Fib_distance
Date,,,,,,,,,
2021-05-16 02:00:00,1.621123e+09,1.0538,1.0164,1.0657,1.0000,292299.347384,301062.256777,1.169975,0.629206
2021-05-16 06:00:00,1.621138e+09,1.0178,1.0183,1.0227,1.0093,32862.129308,33428.219164,1.161918,0.642710
2021-05-16 10:00:00,1.621152e+09,1.0185,1.0082,1.0220,1.0080,47084.905015,47774.247243,1.153953,0.618784
2021-05-16 14:00:00,1.621166e+09,1.0086,1.0359,1.0479,1.0081,112191.202663,114880.672773,1.149146,0.626178
2021-05-16 18:00:00,1.621181e+09,1.0304,0.9756,1.0330,0.9729,243404.839103,244329.577369,1.139794,0.589556


In [83]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_BTC_short = df_BTC.iloc[-2000:]
fig = make_subplots(
    rows=2,
    cols=1,
    subplot_titles=("Historical price", "Fibonacci Distance"),
    shared_xaxes=True,
)

fig.add_trace(
    go.Candlestick(
        name="Historical price",
        x=df_BTC_short.index,
        open=df_BTC_short["Open"],
        high=df_BTC_short["High"],
        low=df_BTC_short["Low"],
        close=df_BTC_short["Close"],
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        name="FibEMA",
        x=df_BTC_short.index,
        y=df_BTC_short["FibEMA"],
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        name="EMA Distance",
        x=df_BTC_short.index,
        y=df_BTC_short["Fib_distance"],
    ),
    row=2,
    col=1,
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    showlegend=True,
    title_text="Historical price and Fibonacci Moving Average",
)
fig.show()


# Fibonacci MA strategy


In [84]:
def buy_func(row: pd.Series, prev_row: pd.Series) -> bool:
    return True if row["FibEMA"] >= row["Close"] else False


def sell_func(row: pd.Series, prev_row: pd.Series, timeframe_count: int) -> bool:
    return True if row["FibEMA"] <= row["Close"] or timeframe_count >= 6 * 4 else False


long_only_backtester(df_BTC, buy_func, sell_func, stop_loss=0.05, take_profit=0.03)


-------------  General informations  -------------
Period: [2021-05-16 02:00:00] -> [2023-03-12 13:00:00]
Intial balance: 1000 $

-------------  Strategy performance  -------------
Final balance: 4258.04 $
Final net balance: 1314.60 $
Strategy net return: 131.46 %
Buy and Hold return: 88.99 %
Strategy winrate: 64.80 %
Strategy fees: 845851.37 $
Strategy volatility: 0.04 %
Sharpe ratio: 0.08 (no risk free rate)
Sharpe ratio: 0.07 (risk free rate = buy and hold)

-------------  Trades informations  --------------
Mean trade duration: 0 days 16:06:01
Total trades: 588
Total good trades: 381
Mean good trades return: 3.19 %
Median good trades return: 3.00 %
Best trades return: 20.94 % | Date: 2022-03-11 05:00:00 | Duration: 0 days 12:00:00
Mean good trade duration: 0 days 13:17:57

Total bad trades: 207
Mean bad trades return: -4.93 %
Median bad trades return: -5.00 %
Worst trades return: -5.00 % | Date: 2021-05-27 02:00:00 | Duration: 0 days 16:00:00
Mean bad trade duration: 0 days 21:15:2

# Fibonacci distance strategy


In [85]:
upper_threshold = 0.7
lower_threshold = 0.3

def buy_func(row: pd.Series, prev_row: pd.Series) -> bool:
    return (
        True
        if (row["FibEMA"] >= row["Close"] and (row["Fib_distance"] <= lower_threshold))
        or (row["FibEMA"] <= row["Close"] and (row["Fib_distance"] >= upper_threshold))
        else False
    )


def sell_func(row: pd.Series, prev_row: pd.Series, timeframe_count: int) -> bool:
    return (
        True
        if (row["FibEMA"] >= row["Close"] and (row["Fib_distance"] >= upper_threshold))
        or (row["FibEMA"] <= row["Close"] and (row["Fib_distance"] <= lower_threshold))
        else False
    )


long_only_backtester(df_BTC, buy_func, sell_func)

-------------  General informations  -------------
Period: [2021-05-16 02:00:00] -> [2023-03-12 13:00:00]
Intial balance: 1000 $

-------------  Strategy performance  -------------
Final balance: 3897.15 $
Final net balance: 3392.37 $
Strategy net return: 339.24 %
Buy and Hold return: 88.99 %
Strategy winrate: 47.83 %
Strategy fees: 44100.76 $
Strategy volatility: 0.36 %
Sharpe ratio: 0.15 (no risk free rate)
Sharpe ratio: 0.15 (risk free rate = buy and hold)

-------------  Trades informations  --------------
Mean trade duration: 4 days 17:20:52
Total trades: 69
Total good trades: 33
Mean good trades return: 21.65 %
Median good trades return: 7.55 %
Best trades return: 235.45 % | Date: 2021-11-13 21:00:00 | Duration: 11 days 12:00:00
Mean good trade duration: 3 days 12:43:38

Total bad trades: 36
Mean bad trades return: -9.23 %
Median bad trades return: -5.53 %
Worst trades return: -39.38 % | Date: 2022-01-25 13:00:00 | Duration: 8 days 16:00:00
Mean bad trade duration: 5 days 19:35:0

## With Stock

In [86]:
import yfinance as yf

ticker = "MSFT" # MSFT, TSLA, AAPL, AMZN, JPM, Gold (GC=F)

stock = yf.Ticker(ticker).history(period='5y',interval="1d")
stock.drop(columns=["Dividends","Stock Splits"],inplace=True)
stock.head()

,Open,High,Low,Close,Volume
Date,,,,,
2018-03-12 00:00:00-04:00,91.074263,91.744342,90.640127,91.329079,26073700
2018-03-13 00:00:00-04:00,91.546141,91.772645,88.686505,89.101768,35387800
2018-03-14 00:00:00-04:00,89.771855,90.045551,88.242938,88.573257,32132000
2018-03-15 00:00:00-04:00,88.271250,89.262216,87.610610,88.884705,27611000
2018-03-16 00:00:00-04:00,89.356571,90.017210,88.639300,89.281067,49081300


In [87]:
stock["FibEMA"] = fibonacci_moving_average(stock.Close)
stock["Fib_distance"] = fibonacci_distance_indicator(stock.Close, stock.FibEMA)

stock.dropna(inplace=True)

stock_short = stock.iloc[-2000:]
fig = make_subplots(
    rows=2,
    cols=1,
    subplot_titles=("Historical price", "Fibonacci Distance"),
    shared_xaxes=True,
)

fig.add_trace(
    go.Candlestick(
        name="Historical price",
        x=stock_short.index,
        open=stock_short["Open"],
        high=stock_short["High"],
        low=stock_short["Low"],
        close=stock_short["Close"],
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        name="FibEMA",
        x=stock_short.index,
        y=stock_short["FibEMA"],
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        name="EMA Distance",
        x=stock_short.index,
        y=stock_short["Fib_distance"],
    ),
    row=2,
    col=1,
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    showlegend=True,
    title_text="Historical price and Fibonacci Moving Average",
)
fig.show()

stock.head()


,Open,High,Low,Close,Volume,FibEMA,Fib_distance
Date,,,,,,,
2018-04-20 00:00:00-04:00,90.517438,90.706190,88.762017,89.658600,31154400,87.879709,0.920112
2018-04-23 00:00:00-04:00,90.357001,90.876080,89.309411,89.988930,22331800,87.991492,0.926802
2018-04-24 00:00:00-04:00,90.828865,91.045936,87.214214,87.884293,34524800,87.918647,0.872386
2018-04-25 00:00:00-04:00,88.054169,88.054169,85.203966,87.119827,33729300,87.804881,0.823948
2018-04-26 00:00:00-04:00,88.290126,89.800164,87.865423,88.960205,42529000,87.866108,0.787218


In [88]:
upper_threshold = 0.8
lower_threshold = 0.2


def buy_func(row: pd.Series, prev_row: pd.Series) -> bool:
    return (
        True
        if (row["FibEMA"] >= row["Close"] and (row["Fib_distance"] <= lower_threshold))
        or (row["FibEMA"] <= row["Close"] and (row["Fib_distance"] >= upper_threshold))
        else False
    )


def sell_func(row: pd.Series, prev_row: pd.Series, timeframe_count: int) -> bool:
    return (
        True
        if (row["FibEMA"] >= row["Close"] and (row["Fib_distance"] >= upper_threshold))
        or (row["FibEMA"] <= row["Close"] and (row["Fib_distance"] <= lower_threshold))
        else False
    )


long_only_backtester(stock, buy_func, sell_func)

-------------  General informations  -------------
Period: [2018-04-20 00:00:00-04:00] -> [2023-03-10 00:00:00-05:00]
Intial balance: 1000 $

-------------  Strategy performance  -------------
Final balance: 2124.67 $
Final net balance: 2047.95 $
Strategy net return: 204.80 %
Buy and Hold return: 277.26 %
Strategy winrate: 63.16 %
Strategy fees: 719.18 $
Strategy volatility: 0.14 %
Sharpe ratio: 0.34 (no risk free rate)
Sharpe ratio: 0.33 (risk free rate = buy and hold)

-------------  Trades informations  --------------
Mean trade duration: 64 days 03:56:50
Total trades: 19
Total good trades: 12
Mean good trades return: 10.07 %
Median good trades return: 4.45 %
Best trades return: 54.55 % | Date: 2020-03-06 00:00:00-05:00 | Duration: 401 days 00:00:00
Mean good trade duration: 73 days 18:05:00

Total bad trades: 7
Mean bad trades return: -4.27 %
Median bad trades return: -3.39 %
Worst trades return: -7.53 % | Date: 2022-04-21 00:00:00-04:00 | Duration: 27 days 00:00:00
Mean bad trade 